# AI‑Powered ETL Pipeline for Personalized Marketing Messages

This notebook demonstrates a complete ETL (Extract, Transform, Load) workflow using Python, Pandas, and the OpenAI API.

The pipeline:
- Loads customer data from a CSV file  
- Masks sensitive information  
- Generates personalized marketing messages using AI  
- Saves the enriched dataset into a new CSV file  

This project began as a bootcamp exercise and was expanded significantly into a full AI‑powered ETL pipeline.

In [29]:
import pandas as pd
from openai import OpenAI

## 📥 Extract Phase

Load the CSV file and convert it into a dictionary structure for easier processing.

In [30]:
df = pd.read_csv("clients.csv")
data = df.to_dict(orient="records")

df.head()

,ID,Name,Account,Card
0,1,Ana Silva,12345,5555-4444-3333-2222
1,2,João Pereira,67890,1111-2222-3333-4444
2,3,Marcos Lima,54321,9999-8888-7777-6666
3,4,Carla Mendes,11223,4444-3333-2222-1111
4,5,Ricardo Souza,99887,2222-1111-4444-3333


## 🔐 Masking Sensitive Data

Mask account and card numbers to simulate real-world data governance practices.

In [31]:
def mask_card(card):
    return "****-****-****-" + card[-4:]

def mask_account(account):
    return "*" * (len(str(account)) - 1) + str(account)[-1:]

for user in data:
    user["Card"] = mask_card(user["Card"])
    user["Account"] = mask_account(str(user["Account"]))

data[:3]

[{'ID': 1,
  'Name': 'Ana Silva',
  'Account': '****5',
  'Card': '****-****-****-2222'},
 {'ID': 2,
  'Name': 'João Pereira',
  'Account': '****0',
  'Card': '****-****-****-4444'},
 {'ID': 3,
  'Name': 'Marcos Lima',
  'Account': '****1',
  'Card': '****-****-****-6666'}]

## 🔄 Transform Phase — AI Message Generation

Use of OpenAI API to generate personalized marketing messages for each customer.

In [32]:
import os
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [33]:
def generate_message(user):
    try:
        completion = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a marketing and banking account manager specialist."},
                {"role": "user", "content": f"Generate a message for {user['Name']} about the importance of investments and savings (200 characters max)."}
            ]
        )
        return completion.choices[0].message.content.strip('"')
    except Exception as e:
        print(f"Error generating message for {user['Name']}: {e}")
        return False

In [34]:
for user in data:
    news = generate_message(user)

    if not news:
        print(f"Skipping user {user['Name']} due to AI failure")
        continue

    user["news"] = [{"description": news}]

data[:3]

[{'ID': 1,
  'Name': 'Ana Silva',
  'Account': '****5',
  'Card': '****-****-****-2222',
  'news': [{'description': 'Hi Ana! Investing and saving are key to financial security. They help grow your wealth over time, provide for future needs, and offer peace of mind. Start today for a brighter tomorrow!'}]},
 {'ID': 2,
  'Name': 'João Pereira',
  'Account': '****0',
  'Card': '****-****-****-4444',
  'news': [{'description': 'Hi João, investing and saving are key to financial security. They help grow wealth and prepare for future needs. Start small, stay consistent, and watch your financial goals come to life!'}]},
 {'ID': 3,
  'Name': 'Marcos Lima',
  'Account': '****1',
  'Card': '****-****-****-6666',
  'news': [{'description': 'Hi Marcos, investing and saving are key to financial security and growth. They can help you achieve goals and prepare for the future. Start small and build your wealth over time!'}]}]

## 📤 Load Phase

Validate the results and save the enriched dataset into a new CSV file.

In [35]:
valid_rows = [u for u in data if u.get("news")]

if not valid_rows:
    print("No valid messages generated. Aborting load phase.")
else:
    output_rows = []

    for user in valid_rows:
        output_rows.append({
            "ID": user["ID"],
            "Name": user["Name"],
            "Account": user["Account"],
            "Card": user["Card"],
            "Message": user["news"][0]["description"]
        })

    df_out = pd.DataFrame(output_rows)
    df_out.to_csv("marketing_messages.csv", index=False, encoding="utf-8")
    df_out.head()

## 🎉 Project Complete

This notebook demonstrates:
- A full ETL workflow  
- Integration with the OpenAI API  
- Data masking best practices  
- Automated generation of marketing messages  

This project is part of my data analytics portfolio.  
Feel free to connect with me on LinkedIn!